# Creating a ReAct (Reasoning and Acting) Agent 

In [1]:
import openai
import re
import httpx
import os
from dotenv import load_dotenv

_ = load_dotenv()
from openai import OpenAI

In [2]:
client = OpenAI()

In [3]:
chat_completion = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[{"role": "user", "content": "Hello world!"}]
)

In [4]:
chat_completion.choices[0].message.content

'Hello! How can I assist you today?'

In [11]:
class Agent:
    # Parameterized by a system message to allow user to pass that in and then save that as an attribute.
    # Keep track of the messages that have been sent and received.
    def __init__(self, system=""):
        self.system = system
        self.messages = []
        if self.system:
            self.messages.append({"role": "system", "content": system})
    
    def __call__(self, message):
        self.messages.append({"role": "user", "content": message})
        result = self.execute()
        self.messages.append({"role": "assistant", "content": result})
        return result
    
    def execute(self):
        completion = client.chat.completions.create(
            model="gpt-4o",
            temperature=0,
            messages=self.messages
        )
        return completion.choices[0].message.content




In [12]:
prompt = """
You run in a loop of Thought, Action, PAUSE, Observation.
At the end of the loop you output an Answer
Use Thought to describe your thoughts about the question you have been asked.
Use Action to run one of the actions available to you - then return PAUSE.
Observation will be the result of running those actions.

Your available actions are:

calculate:
e.g. calculate: 4 * 7 / 3
Runs a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary

average_dog_weight:
e.g. average_dog_weight: Collie
returns average weight of a dog when given the breed

Example session:

Question: How much does a Bulldog weigh?
Thought: I should look the dogs weight using average_dog_weight
Action: average_dog_weight: Bulldog
PAUSE

You will be called again with this:

Observation: A Bulldog weights 51 lbs

You then output:

Answer: A bulldog weights 51 lbs
""".strip()


In [13]:
# The following are tools for the agent to use
# A function to calculate the result of a string
def calculate(what):
    return eval(what)
# A function to return the average weight of a dog given the breed
def average_dog_weight(name):
    if name in "Scottish Terrier": 
        return("Scottish Terriers average 20 lbs")
    elif name in "Border Collie":
        return("a Border Collies average weight is 37 lbs")
    elif name in "Toy Poodle":
        return("a toy poodles average weight is 7 lbs")
    else:
        return("An average dog weights 50 lbs")
# Creating a dictionary by mapping names to the known actions
known_actions = {
    "calculate": calculate,
    "average_dog_weight": average_dog_weight
}

In [14]:
# Initiate the agent
abot = Agent(prompt)

In [15]:
result = abot("How much does a toy poodle weigh?")
print(result)

Thought: I should look up the average weight of a toy poodle using the average_dog_weight action.
Action: average_dog_weight: Toy Poodle
PAUSE


In [16]:
result = average_dog_weight("Toy Poodle")

In [17]:
result

'a toy poodles average weight is 7 lbs'

In [18]:
next_prompt = "Observation: {}".format(result)

In [19]:
abot(next_prompt)

'Answer: A toy poodle weighs an average of 7 lbs.'

In [20]:
abot.messages

[{'role': 'system',
  'content': 'You run in a loop of Thought, Action, PAUSE, Observation.\nAt the end of the loop you output an Answer\nUse Thought to describe your thoughts about the question you have been asked.\nUse Action to run one of the actions available to you - then return PAUSE.\nObservation will be the result of running those actions.\n\nYour available actions are:\n\ncalculate:\ne.g. calculate: 4 * 7 / 3\nRuns a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary\n\naverage_dog_weight:\ne.g. average_dog_weight: Collie\nreturns average weight of a dog when given the breed\n\nExample session:\n\nQuestion: How much does a Bulldog weigh?\nThought: I should look the dogs weight using average_dog_weight\nAction: average_dog_weight: Bulldog\nPAUSE\n\nYou will be called again with this:\n\nObservation: A Bulldog weights 51 lbs\n\nYou then output:\n\nAnswer: A bulldog weights 51 lbs'},
 {'role': 'user', 'content': 'How much does a 

In [21]:
#Reinitialize the agent

abot = Agent(prompt)

In [23]:
question = """I have two dogs, a border collie and a scottish terrier. \
How much do they weigh together?"""
abot(question)

'Thought: I need to find the average weight of both a Border Collie and a Scottish Terrier, then sum these weights to get the total weight of both dogs together.\nAction: average_dog_weight: Border Collie\nPAUSE'

In [24]:
next_prompt = "Observation: {}".format(average_dog_weight("Border Collie"))
print(next_prompt)

Observation: a Border Collies average weight is 37 lbs


In [25]:
abot(next_prompt)

'Thought: Now I need to find the average weight of a Scottish Terrier and then sum it with the average weight of a Border Collie.\nAction: average_dog_weight: Scottish Terrier\nPAUSE'

In [26]:
next_prompt = "Observation: {}".format(average_dog_weight("Scottish Terrier"))
print(next_prompt)

Observation: Scottish Terriers average 20 lbs


In [27]:
abot(next_prompt)

'Thought: I now have the average weights of both a Border Collie (37 lbs) and a Scottish Terrier (20 lbs). I need to sum these weights to find the total weight of both dogs together.\nAction: calculate: 37 + 20\nPAUSE'

In [28]:
next_prompt = "Observation: {}".format(eval("37 + 20"))
print(next_prompt)

Observation: 57


In [29]:
abot(next_prompt)

'Answer: Together, a Border Collie and a Scottish Terrier weigh approximately 57 lbs.'

In [32]:
# Create regex to parse the action and the argument
action_re = re.compile(r'^Action: (\w+): (.*)$') 

In [33]:
def query(question, max_turns=5):
    # Number of iterations
    i = 0
    # Initiate the agent with the prompt
    bot = Agent(prompt)
    # Query question 
    next_prompt = question
    # Controls how many iterations the agent will run
    while i < max_turns:
        i += 1
        result = bot(next_prompt)
        print(result)
        # Use regex to parse the response from the agent and get a list of actions
        actions = [
            action_re.match(a) 
            for a in result.split('\n') 
            if action_re.match(a)
        ]
        if actions:
            # There is an action to run
            action, action_input = actions[0].groups()
            if action not in known_actions:
                raise Exception("Unknown action: {}: {}".format(action, action_input))
            print(" -- running {} {}".format(action, action_input))
            observation = known_actions[action](action_input)
            print("Observation:", observation)
            next_prompt = "Observation: {}".format(observation)
        else:
            return

In [34]:
question = """I have 2 dogs, a border collie and a scottish terrier. \
What is their combined weight"""
query(question)

Thought: I need to find the average weight of both a Border Collie and a Scottish Terrier, then sum these weights to get the combined weight.
Action: average_dog_weight: Border Collie
PAUSE
 -- running average_dog_weight Border Collie
Observation: a Border Collies average weight is 37 lbs
Thought: Now I need to find the average weight of a Scottish Terrier.
Action: average_dog_weight: Scottish Terrier
PAUSE
 -- running average_dog_weight Scottish Terrier
Observation: Scottish Terriers average 20 lbs
Thought: I now have the average weights of both dogs. I need to sum these weights to get the combined weight.
Action: calculate: 37 + 20
PAUSE
 -- running calculate 37 + 20
Observation: 57
Answer: The combined weight of a Border Collie and a Scottish Terrier is 57 lbs.
